<a href="https://colab.research.google.com/github/qcjgdejesus/CPE-310-2526A/blob/main/Netflix_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import os

DATA_PATH = "netflix_titles.csv"

In [ ]:
df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
display(df.head())
display(df.tail())

print("Columns:", list(df.columns))

Shape: (8807, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
8806,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


Columns: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']


In [ ]:
print("Missing values per column:")
print(df.isnull().sum().sort_values(ascending=False))

Missing values per column:
director        2634
country          831
cast             825
date_added        10
rating             4
duration           3
show_id            0
type               0
title              0
release_year       0
listed_in          0
description        0
dtype: int64


In [ ]:
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce').astype('Int64')
print(df.dtypes)

show_id                 object
type                    object
title                   object
director                object
cast                    object
country                 object
date_added      datetime64[ns]
release_year             Int64
rating                  object
duration                object
listed_in               object
description             object
dtype: object


In [ ]:
print("Type counts:\n", df['type'].value_counts())
print("Top 5 ratings:\n", df['rating'].value_counts().head(5))
print("Most frequent release year:", int(df['release_year'].mode().dropna().iat[0]))

Type counts:
 type
Movie      6131
TV Show    2676
Name: count, dtype: int64
Top 5 ratings:
 rating
TV-MA    3207
TV-14    2160
TV-PG     863
R         799
PG-13     490
Name: count, dtype: int64
Most frequent release year: 2018


In [ ]:
df['duration'] = df['duration'].fillna('')

def parse_duration(x):
    x = str(x).strip()
    if x == '':
        return np.nan, np.nan
    if 'min' in x.lower():
        try:
            mins = int(''.join([c for c in x if c.isdigit()]))
            return mins, np.nan
        except:
            return np.nan, np.nan
    if 'season' in x.lower():
        try:
            num = int(''.join([c for c in x if c.isdigit()]))
            return np.nan, num
        except:
            return np.nan, np.nan
    try:
        val = int(''.join([c for c in x if c.isdigit()]))
        return val, np.nan
    except:
        return np.nan, np.nan

parsed = df['duration'].apply(parse_duration)
df['duration_minutes'] = parsed.apply(lambda t: t[0])
df['num_seasons'] = parsed.apply(lambda t: t[1])

print("Average movie length (min):", df.loc[df['type']=='Movie','duration_minutes'].dropna().mean())
print("Average seasons (TV):", df.loc[df['type']=='TV Show','num_seasons'].dropna().mean())

Average movie length (min): 99.57718668407311
Average seasons (TV): 1.764947683109118


In [ ]:
# 6. Genre analysis (split 'listed_in')
df_genres = df[['title','release_year','listed_in']].dropna(subset=['listed_in']).copy()
df_genres_exploded = df_genres.assign(genre=df_genres['listed_in'].str.split(',')).explode('genre')
df_genres_exploded['genre'] = df_genres_exploded['genre'].str.strip()
genre_avg_year = df_genres_exploded.dropna(subset=['release_year']).groupby('genre')['release_year'].mean().sort_values(ascending=False)
print("Genres sorted by average release year (top 10):")
print(genre_avg_year.head(10))

Genres sorted by average release year (top 10):
genre
TV Mysteries                    2018.346939
TV Horror                            2018.2
Reality TV                      2017.894118
Stand-Up Comedy & Talk Shows    2017.857143
TV Thrillers                    2017.736842
Crime TV Shows                  2017.687234
Spanish-Language TV Shows       2017.477011
TV Action & Adventure           2017.404762
Docuseries                      2017.232911
TV Dramas                       2017.190039
Name: release_year, dtype: Float64


In [ ]:
# 7. Temporal trends: release_year vs year_added
df['year_added'] = df['date_added'].dt.year
df['acquisition_lag_years'] = df['year_added'] - df['release_year']
lag_mask = df['acquisition_lag_years'].between(-100,100)
lag_clean = df.loc[lag_mask & df['acquisition_lag_years'].notna()]
print("Mean acquisition lag:", lag_clean['acquisition_lag_years'].mean())
print("Median acquisition lag:", lag_clean['acquisition_lag_years'].median())

# Plot simple time series charts and save them
plt.figure(figsize=(10,4))
df['release_year'].dropna().astype(int).value_counts().sort_index().plot()
plt.title('Counts by release_year')
plt.tight_layout()
plt.savefig('release_year_timeseries.png')
plt.close()

plt.figure(figsize=(10,4))
df['year_added'].dropna().astype(int).value_counts().sort_index().plot()
plt.title('Counts by year_added')
plt.tight_layout()
plt.savefig('year_added_timeseries.png')
plt.close()

Mean acquisition lag: 4.690894476977839
Median acquisition lag: 1.0


In [ ]:
# 8. Rating vs Type - crosstab
type_rating = pd.crosstab(df['type'], df['rating'], normalize='index').round(3)
display(type_rating)

rating,66 min,74 min,84 min,G,NC-17,NR,PG,PG-13,R,TV-14,TV-G,TV-MA,TV-PG,TV-Y,TV-Y7,TV-Y7-FV,UR
type,,,,,,,,,,,,,,,,,
Movie,0.0,0.0,0.0,0.007,0.0,0.012,0.047,0.08,0.130,0.233,0.021,0.336,0.088,0.021,0.023,0.001,0.0
TV Show,0.0,0.0,0.0,0.000,0.0,0.002,0.000,0.00,0.001,0.274,0.035,0.428,0.121,0.066,0.073,0.000,0.0


In [ ]:
# 9. Filtering: R-rated titles added after 2020
r_after_2020 = df[(df['rating']=='R') & (df['year_added']>2020)]
print("R-rated titles added after 2020:", len(r_after_2020))
display(r_after_2020[['title','release_year','date_added','country']].head(200))

R-rated titles added after 2020: 190


,title,release_year,date_added,country
46,Safe House,2012,2021-09-16,"South Africa, United States, Japan"
48,Training Day,2001,2021-09-16,United States
81,Kate,2021,2021-09-10,United States
122,In the Cut,2003,2021-09-02,"United Kingdom, Australia, France, United States"
131,Blade Runner: The Final Cut,1982,2021-09-01,United States
...,...,...,...,...
8072,Spring Breakers,2012,2021-01-10,"United States, France"
8106,Striptease,1996,2021-01-01,United States
8126,Superbad,2007,2021-01-01,United States
8272,The Departed,2006,2021-01-01,"United States, Hong Kong"


In [ ]:
# 10. Aggregations & GroupBy: by country avg release_year
df_countries = df[['title','release_year','country']].dropna(subset=['country']).copy()
df_countries_exploded = df_countries.assign(country=df_countries['country'].str.split(',')).explode('country')
df_countries_exploded['country'] = df_countries_exploded['country'].str.strip()
country_avg_release = df_countries_exploded.dropna(subset=['release_year']).groupby('country')['release_year'].mean().sort_values(ascending=False)
display(country_avg_release.head(50).reset_index().rename(columns={'release_year':'avg_release_year'}))

# Proportion Movies vs TV Shows by year_added
year_type_counts = df.dropna(subset=['year_added']).groupby(['year_added','type']).size().unstack(fill_value=0)
year_type_prop = year_type_counts.div(year_type_counts.sum(axis=1), axis=0).round(3)
display(year_type_prop.reset_index())

,country,avg_release_year
0,Angola,2020.0
1,Cameroon,2020.0
2,Mauritius,2020.0
3,Cayman Islands,2019.5
4,Palestine,2019.0
5,Malawi,2019.0
6,Mozambique,2019.0
7,Iceland,2018.272727
8,Albania,2018.0
9,Guatemala,2018.0


type,year_added,Movie,TV Show
0,2008.0,0.500,0.500
1,2009.0,1.000,0.000
2,2010.0,1.000,0.000
3,2011.0,1.000,0.000
4,2012.0,1.000,0.000
5,2013.0,0.600,0.400
6,2014.0,0.826,0.174
7,2015.0,0.767,0.233
8,2016.0,0.605,0.395
9,2017.0,0.721,0.279


In [ ]:
# 11. Reusable functions
def titles_by_director(director_name):
    mask = df['director'].notna() & df['director'].str.contains(director_name, case=False, na=False)
    subset = df.loc[mask].sort_values('release_year', ascending=False)
    return subset[['title','type','release_year','date_added','rating','duration']]

def plot_top_n(df_input, column, n=10, title=None, savepath=None):
    counts = df_input[column].dropna().astype(str).value_counts().head(n)
    plt.figure(figsize=(8,5))
    counts.plot(kind='bar')
    plt.title(title or f"Top {n} categories in {column}")
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.tight_layout()
    if savepath:
        plt.savefig(savepath)
        plt.close()
    else:
        plt.show()

# Example usage:
print("Titles with director containing 'Steven Spielberg':", len(titles_by_director('Steven Spielberg')))
plot_top_n(df_genres_exploded, 'genre', n=10, title='Top 10 genres', savepath='top10_genres.png')

Titles with director containing 'Steven Spielberg': 11
